# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902085


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/104 [00:00<?, ?it/s]

Rendering models:   2%|▏         | 2/104 [00:05<04:41,  2.76s/it]

Rendering models:   3%|▎         | 3/104 [00:05<03:18,  1.96s/it]

Rendering models:   5%|▍         | 5/104 [00:05<02:18,  1.40s/it]

Rendering models:   6%|▌         | 6/104 [00:07<02:26,  1.49s/it]

...

Rendering models:  86%|████████▌ | 89/104 [00:13<00:00, 18.09it/s]

Rendering models:  89%|████████▉ | 93/104 [00:13<00:00, 19.31it/s]

Rendering models:  93%|█████████▎| 97/104 [00:13<00:00, 21.99it/s]

Rendering models:  99%|█████████▉| 103/104 [00:14<00:00, 24.35it/s]

equidad1                              0.077407
ElisabethB                            0.000335
not-logged-in-03b60317d6ee47942c58    0.000671
Thedunkmasta                          0.002346
Thedunkmasta                          0.001595
Thedunkmasta                          0.001763
Thedunkmasta                          0.001911
Thedunkmasta                          0.002457
Thedunkmasta                          0.003096
Thedunkmasta                          0.002019
Thedunkmasta                          0.002428
Thedunkmasta                          0.001685
Thedunkmasta                          0.003550
mlang2017                             0.008855
rcbeck                                0.000686
sinjinza1984                          0.000211
JEBala                                0.000574
not-logged-in-aac1a66e38768e0fe308    0.001276
not-logged-in-ea8ed61a7274cfb6a43b    0.044619
stekenxu                              0.000240
msross                                0.000549
clizardo     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())